In [ ]:
# Reference
'''
https://arxiv.org/abs/1608.06993
https://github.com/liuzhuang13/DenseNet
https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html
https://pytorch.org/docs/stable/torchvision/models.html
https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
https://pytorch.org/tutorials/advanced/neural_style_tutorial.html
https://pytorch.org/tutorials/beginner/saving_loading_models.html
'''

In [1]:
import numpy as np 
import pandas as pd 
import os
import json
import random
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
import time
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from PIL import Image 

In [2]:
# shuffle 
seed = 34
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = True
use_gpu = True

# data agumentation and normalization for training
trans_train = transforms.Compose([transforms.RandomResizedCrop(size=224),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomRotation(30),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

trans_valid = transforms.Compose([transforms.Resize(size=224),
                                  transforms.CenterCrop(size=224),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# inputs
batch_size = 128
num_epochs = 20
stops = 5

# load data
train_path = 'C:/Users/HP/Desktop/ai_challenger_pdr2018_trainingset_20181023/AgriculturalDisease_trainingset/images'
val_path = 'C:/Users/HP/Desktop/Test_images'

train=pd.read_json('C:/Users/HP/Desktop/ai_challenger_pdr2018_trainingset_20181023/AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations.json')
print(len(train))
train.head()

31718


,disease_class,image_id
0,1,62fd8bf4d53a1b94fbac16738406f10b.jpg
1,1,0bdec5cccbcade6b6e94087cb5509d98.jpg
2,1,8951e940341f77c8d361c1872c67b16d.jpg
3,1,7ed158da58c451f75fb790530d6f19cc.jpg
4,1,9b7399aa-1c3c-4137-ae4e-196cd23fe573___FREC_Sc...


In [3]:
n_train = len(train)
categories = set(train['disease_class'])
n_class = len(categories)
print('disease_class:{}\n{}'.format(n_class,categories))

number_of_classes = []
for i in range(n_class):
    number_of_classes.append(list(train['disease_class']).count(i))
print(number_of_classes)

val = pd.read_json('C:/Users/HP/Desktop/validation_labels.json')
n_val = len(val)
number_of_classes = []
for i in range(n_class):
    number_of_classes.append(list(val['disease_class']).count(i))
print(number_of_classes)

disease_class:61
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60}
[1185, 211, 152, 427, 142, 40, 598, 116, 110, 376, 191, 167, 483, 355, 208, 498, 815, 294, 381, 462, 503, 419, 61, 630, 367, 1828, 1799, 251, 857, 770, 1025, 287, 377, 1430, 203, 510, 251, 446, 242, 192, 583, 1208, 319, 966, 1, 1, 251, 442, 264, 1109, 325, 336, 43, 22, 421, 807, 542, 271, 1414, 2473, 261]
[169, 30, 22, 61, 20, 6, 85, 12, 18, 54, 27, 24, 69, 51, 29, 71, 116, 42, 54, 66, 74, 59, 9, 90, 52, 269, 262, 36, 122, 110, 147, 40, 54, 204, 29, 73, 36, 64, 35, 27, 83, 173, 46, 138, 1, 0, 36, 63, 38, 158, 46, 48, 4, 5, 60, 115, 77, 39, 202, 353, 37]


In [4]:
num_class = n_class
model = models.densenet201()

# set the .requires_grad attribute of the parameters 
print("Params to learn:")
for para in list(model.parameters()):
    para.requires_grad=False
for para in list(model.features.denseblock3.parameters()):
    para.requires_grad=True
for para in list(model.features.transition3.parameters()):
    para.requires_grad=True
for para in list(model.features.denseblock4.parameters()):
    para.requires_grad=True
for para in list(model.features.norm5.parameters()):
    para.requires_grad=True

model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(1920, num_class),
)

C:\Users\HP\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


Params to learn:


In [ ]:
# put the model to a GPU
device = torch.device("cuda:0")
model.to(device)

In [ ]:
# create model and dataparallel
cuda_device_ids = [0,1]
model = model.cuda(cuda_device_ids[0])
model = nn.DataParallel(model, device_ids=cuda_device_ids)
model.load_state_dict(torch.load('C:/Users/HP/Desktop/densenet201.pth'))

In [ ]:
# create a DenseNet model
class Model(Dataset):
    def __init__(self, df_data, data_dir = './', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        label,img_name = self.df[index]
        img_path = os.path.join(self.data_dir, img_name)
        with Image.open(img_path) as img:
            image = img.convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, label

dataset_train = Model(df_data=train, 
    data_dir=train_path, transform=trans_train)
dataset_valid = Model(df_data=val, 
    data_dir=val_path, transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=64, shuffle=False, num_workers=0)

In [ ]:
# optimize the parameters
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

In [ ]:
# create a cross-entropy loss function that accepts soft target
def cross_entropy(input, target, size_average=True):
    logsoftmax = nn.LogSoftmax()
    if size_average:
        return torch.mean(torch.sum(-target * logsoftmax(input), dim=1))
    else:
        return torch.sum(torch.sum(-target * logsoftmax(input), dim=1))
    
criterion = cross_entropy
optimizer = optim.SGD(params_to_update,lr = 1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
label_smoothing = 0.1

In [ ]:
# training
best_val_acc = 0.875
best_epoch = 0
epoch_since = 0

for epoch in range(num_epochs):
    since = time.time()
    
    # schedule the learning rate
    scheduler.step()
    model.train() # Set model to training mode
    
    train_total_samples = 0
    train_acc = 0
    train_loss = 0
    
    # iterate over data.
    for i, data in enumerate(loader_train):
        print('.',end='')        
        # get the inputs
        inputs, label = data
        train_total_samples += label.size()[0]        
        labels = label.resize(label.size()[0], 1)
        # convert labels to a one-hot variable
        labels = torch.FloatTensor(label.size()[0], n_class).zero_().scatter_(1, labels.resize(label.size()[0], 1) ,1)
        labels = (1 - label_smoothing) * labels + (label_smoothing / n_class)
        if use_gpu:
            inputs, labels, label = inputs.cuda(), labels.cuda(), label.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = model(inputs)        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # statistics
        train_pred = torch.argmax(outputs.data, dim=1)        
        train_acc += torch.sum(train_pred == label.data)
        train_loss += loss.item() * labels.size()[0]
            
    model.eval()
    valid_total_samples = 0
    valid_acc = 0
    val_loss = 0 
    
    for _, data in enumerate(loader_valid):     
        inputs, label = data
        valid_total_samples += label.size()[0]        
        labels = label
        labels = torch.FloatTensor(label.size()[0], n_class).zero_().scatter_(1, labels.resize(label.size()[0], 1) ,1)
        labels = (1 - label_smoothing) * labels + (label_smoothing / n_class)
        if use_gpu:
            inputs, labels, label = inputs.cuda(), labels.cuda(), label.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        valid_pred = torch.argmax(outputs.data, dim=1)        
        valid_acc += torch.sum(valid_pred == label.data)
        val_loss += loss.item() * labels.size()[0]
    
    # accuaracy and loss
    train_acc = train_acc.cpu().numpy() / train_total_samples
    valid_acc = valid_acc.cpu().numpy() / valid_total_samples
    train_loss = train_loss / train_total_samples
    val_loss = val_loss / valid_total_samples
    print()
    
    time_elapsed = time.time()
    print('[Epoch %d] train loss %.6f train acc %.6f  valid loss %.6f valid acc %.6f  time %.6f' % (
        epoch, train_loss, train_acc, val_loss, valid_acc,time_elapsed-since))
    
    # save the best model
    if valid_acc > best_val_acc:
        best_val_acc = valid_acc
        best_epoch = epoch
        epoch_since = 0
        print('save model...')
        torch.save(model.state_dict(), 'tuned-densenet201.pth')
        print('saved.')
    else:
        epoch_since += 1
        
    if epoch_since > stops:
        break
            
print('Finished Training')
print('best_epoch: %d, best_val_acc %.6f' % (best_epoch, best_val_acc))